In [12]:
#解析html,获取二手房信息
import re
import requests
from  bs4 import BeautifulSoup 
import get_allurls
from get_allurls import *
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
from numpy import random

In [13]:
def parse_url():
    info_db=DataFrame()
    all_urls=get_allurls.get_allurl()
    
    for url in all_urls:
        info={}
        
        res=requests.get(url)#allow_redirects=False关闭重定向
        if res.status_code==200:
            html=res.content
            soup=BeautifulSoup(html,'html.parser')
            #该网站的html代码中有的标识为空，字典赋值出现报错IndexError: list index out of range
            #加上try.....except 错误机制跳过空值
            try:
                #获取标题
                title=soup.select('.main')[0].text
                #获取房屋总价(单位万)
                total_price=soup.select('.total')[0].text#输出如：'775'
                #获取房屋单价(单位元)
                price=soup.select('.unitPriceValue')[0].text#输出如：'85147元/平米'
                #获取房屋名称及所属区
                s=soup.find_all('div',attrs={'class':'fl l-txt'})[0]
                ##返回的s格式如下
                ##<div class="fl l-txt"><a href="/">上海房产网</a><span class="stp"> &gt; </span><a href="/ershoufang/">上海二手房</a><span class="stp"> &gt; </span><a href="/ershoufang/jingan/">静安二手房</a><span class="stp"> &gt; </span><a href="/ershoufang/daning/">大宁二手房</a><span class="stp"> &gt; </span><a href="/ershoufang/c5011000010670/">上海滩大宁城(公寓)二手房</a><span class="stp"> &gt; </span> <span>当前房源</span> </div>
                s=re.findall('<a.*?</a',str(s))
                ##返回的s格式如下
                ##['<a href="/">上海房产网</a',
                ##'<a href="/ershoufang/">上海二手房</a',
                ##'<a href="/ershoufang/jingan/">静安二手房</a',
                ##'<a href="/ershoufang/daning/">大宁二手房</a',
                ##'<a href="/ershoufang/c5011000010670/">上海滩大宁城(公寓)二手房</a']
                name=re.findall('>(.*?)<',s[-1])[0]#将s继续规整，得到name,'上海滩大宁城(公寓)二手房'
                district=re.findall('>(.*?)<',s[2])[0]#'静安二手房'
                info['title']=title#标题
                info['name']=name#小区名称
                info['district']=district#所在区
                info['total_price']=total_price#房屋总价
                info['price']=total_price#单价
                      
                #获取房屋基本信息
                basic_info=re.findall("""房屋户型</span>.*?</li>|
                                     所在楼层</span>.*?</li>|
                                     建筑面积</span>.*?</li>|
                                     户型结构</span>.*?</li>|
                                     套内面积</span>.*?</li>|
                                     建筑类型</span>.*?</li>|
                                     房屋朝向</span>.*?</li>|
                                     建筑结构</span>.*?</li>|
                                     装修情况</span>.*?</li>|
                                     梯户比例</span>.*?</li>|
                                     配备电梯</span>.*?</li>
                                     """,str(soup('li')),re.VERBOSE)
            #输出的basic_info列表格式如下：
            #['房屋户型</span>3室2厅1厨1卫</li>',
            #'所在楼层</span>中楼层 (共23层)</li>',
            #'建筑面积</span>91.02㎡</li>',
            #'户型结构</span>暂无数据</li>',
            #'套内面积</span>暂无数据</li>',
            #'建筑类型</span>板楼</li>',
            #'房屋朝向</span>南</li>',
            #'建筑结构</span>钢混结构</li>',
            #'装修情况</span>精装</li>',
            #'梯户比例</span>两梯三户</li>',
            #'配备电梯</span>有</li>']

                basic_info_list=[]
                for i in basic_info:
                    i=re.findall('>(.*?)<',i)[0]#对basic_info列表中的值进行规整
                    basic_info_list.append(i)
                #规整后的basic_info_list格式如下：
                #['3室2厅1厨1卫',
                #'中楼层 (共23层)',
                #'91.02㎡',
                #'暂无数据',
                #'暂无数据',
                #'板楼',
                #'南',
                #'钢混结构',
                #'精装',
                #'两梯三户',
                #'有']
                info['house_type']=basic_info_list[0]#房屋户型
                info['floor_loc']=basic_info_list[1]#所在楼层
                info['house_area']=basic_info_list[2]#建筑面积
                info['house_struc']=basic_info_list[3]#户型结构
                info['real_area']=basic_info_list[4]#套内面积
                info['archi_cate']=basic_info_list[5]#建筑类型
                info['house_toward']=basic_info_list[6]#房屋朝向
                info['archi_struc']=basic_info_list[7]#建筑结构
                info['deco_condition']=basic_info_list[8]#装修情况
                info['elevator_rate']=basic_info_list[9]#梯户比例
                info['elevator_equip']=basic_info_list[10]#配备电梯

            except IndexError:
                continue             
            info_db=pd.concat([info_db,DataFrame(info,index=[1])],axis=0,ignore_index=True)#将字典转为DataFrame，循环concat
            
    return(info_db)

In [14]:
data=parse_url()


请输入生成页数：100


In [15]:
data.describe()

,title,name,district,total_price,price,house_type,floor_loc,house_area,house_struc,real_area,archi_cate,house_toward,archi_struc,deco_condition,elevator_rate,elevator_equip
count,2978,2978,2978,2978,2978,2978,2978,2978,2978,2978,2978,2978,2978,2978,2978,2978
unique,2973,1972,15,687,687,50,122,2494,5,187,4,35,6,4,68,3
top,森兰绿地的法式精装+额度有放心用+ 详情请来电咨询,上海康城二手房,浦东二手房,310,310,2室1厅1厨1卫,高楼层 (共6层),85㎡,平层,暂无数据,板楼,南,钢混结构,精装,一梯两户,无
freq,2,10,766,28,28,785,579,6,2883,2768,2741,2158,1893,1684,1104,1551


In [ ]:
pd.save